In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

os.environ["CODECARBON_DISABLE"] = "1"
model_id = "mistralai/Mistral-7B-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map={'': 0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding='max_length')

/home/ksaff/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/model.safetensors.index.json
['/home/ksaff/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/model-00001-of-00002.safetensors', '/home/ksaff/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/model-00002-of-00002.safetensors']



  0%|          | 0/200 [07:47<?, ?it/s]


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[codecarbon INFO @ 15:35:53] Energy consumed for RAM : 0.001267 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:35:53] Energy consumed for all GPUs : 0.005735 kWh. Total GPU Power : 138.66 W
[codecarbon INFO @ 15:35:53] Energy consumed for all CPUs : 0.004604 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:35:53] 0.011606 kWh of electricity used since the beginning.


In [32]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [33]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [34]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3758886912 || trainable%: 0.18132346515244138


In [35]:
import transformers
from datasets import load_dataset

data = load_dataset("/home/ksaff/Desktop/ttyd/fine_tuning/dataset/")

In [36]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map: 100%|██████████| 1051/1051 [00:00<00:00, 41539.50 examples/s]


In [38]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

[codecarbon INFO @ 15:36:26] [setup] RAM Tracking...
[codecarbon INFO @ 15:36:26] [setup] GPU Tracking...
[codecarbon INFO @ 15:36:26] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:36:26] [setup] CPU Tracking...
[codecarbon WARNING @ 15:36:26] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 15:36:27] We saw that you have a 13th Gen Intel(R) Core(TM) i5-13600KF but we don't know it. Please contact us.
[codecarbon INFO @ 15:36:27] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i5-13600KF
[codecarbon INFO @ 15:36:27] >>> Tracker's metadata:
[codecarbon INFO @ 15:36:27]   Platform system: Linux-6.2.0-39-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 15:36:27]   Python version: 3.10.13
[codecarbon INFO @ 15:36:27]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 15:36:27]   Available RAM : 31.196 GB
[codecarbon INFO @ 15:36:27]   CPU count: 20
[codecarbon INFO @ 15:36:27]   CPU model: 13th Gen Intel(R) Core(TM) i5-13600KF
[code

{'loss': 2.7518, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


[codecarbon INFO @ 15:36:34] Energy consumed for RAM : 0.001657 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:36:34] Energy consumed for all GPUs : 0.009599 kWh. Total GPU Power : 347.45500000000004 W
[codecarbon INFO @ 15:36:34] Energy consumed for all CPUs : 0.006021 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:36:34] 0.017277 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:36:34] Energy consumed for RAM : 0.000097 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:36:34] Energy consumed for all GPUs : 0.001995 kWh. Total GPU Power : 348.754 W
[codecarbon INFO @ 15:36:34] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:36:34] 0.002446 kWh of electricity used since the beginning.
                                               
  1%|          | 2/200 [00:06<10:14,  3.10s/it]

{'loss': 2.6842, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}


[codecarbon INFO @ 15:36:38] Energy consumed for RAM : 0.001413 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:36:38] Energy consumed for all GPUs : 0.007965 kWh. Total GPU Power : 353.362 W
[codecarbon INFO @ 15:36:38] Energy consumed for all CPUs : 0.005135 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:36:38] 0.014513 kWh of electricity used since the beginning.
                                               
  2%|▏         | 3/200 [00:09<10:10,  3.10s/it]

...

{'loss': 0.1845, 'learning_rate': 2.0000000000000003e-06, 'epoch': 3.03}


[codecarbon INFO @ 15:46:23] Energy consumed for RAM : 0.003314 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:46:23] Energy consumed for all GPUs : 0.065128 kWh. Total GPU Power : 350.932 W
[codecarbon INFO @ 15:46:23] Energy consumed for all CPUs : 0.012042 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:46:23] 0.080483 kWh of electricity used since the beginning.
                                                 
                                                 
100%|██████████| 200/200 [09:55<00:00,  2.97s/it][codecarbon INFO @ 15:46:25] Energy consumed for RAM : 0.001934 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:46:25] Energy consumed for all GPUs : 0.058098 kWh. Total GPU Power : 349.83400000000006 W
[codecarbon INFO @ 15:46:25] Energy consumed for all CPUs : 0.007027 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:46:25] 0.067059 kWh of electricity used since the beginning.
/home/ksaff/miniconda3/envs/llm_jupyter/lib/python3.10/site-packages

{'loss': 0.2046, 'learning_rate': 0.0, 'epoch': 3.04}
{'train_runtime': 595.2406, 'train_samples_per_second': 5.376, 'train_steps_per_second': 0.336, 'train_loss': 0.6169997519999743, 'epoch': 3.04}


TrainOutput(global_step=200, training_loss=0.6169997519999743, metrics={'train_runtime': 595.2406, 'train_samples_per_second': 5.376, 'train_steps_per_second': 0.336, 'train_loss': 0.6169997519999743, 'epoch': 3.04})

[codecarbon INFO @ 15:46:31] Energy consumed for RAM : 0.003850 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:46:31] Energy consumed for all GPUs : 0.067800 kWh. Total GPU Power : 131.986 W
[codecarbon INFO @ 15:46:31] Energy consumed for all CPUs : 0.013989 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:46:31] 0.085640 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:46:34] Energy consumed for RAM : 0.003606 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:46:34] Energy consumed for all GPUs : 0.067346 kWh. Total GPU Power : 132.439 W
[codecarbon INFO @ 15:46:34] Energy consumed for all CPUs : 0.013104 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:46:34] 0.084056 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:46:34] Energy consumed for RAM : 0.002047 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:46:34] Energy consumed for all GPUs : 0.059707 kWh. Total GPU Power : 134.12 W
[codecarbon INFO @ 15:46:

In [ ]:
trainer.save_model("1st_try")

[codecarbon INFO @ 15:57:01] Energy consumed for RAM : 0.005897 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:57:01] Energy consumed for all GPUs : 0.094413 kWh. Total GPU Power : 130.462 W
[codecarbon INFO @ 15:57:01] Energy consumed for all CPUs : 0.021427 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:57:01] 0.121737 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:57:04] Energy consumed for RAM : 0.005653 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:57:04] Energy consumed for all GPUs : 0.093262 kWh. Total GPU Power : 132.933 W
[codecarbon INFO @ 15:57:04] Energy consumed for all CPUs : 0.020541 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:57:04] 0.119457 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:57:04] Energy consumed for RAM : 0.004094 kWh. RAM Power : 11.698551177978516 W
[codecarbon INFO @ 15:57:04] Energy consumed for all GPUs : 0.085377 kWh. Total GPU Power : 132.51 W
[codecarbon INFO @ 15:57: